In [0]:
# initial set up to store the datasets into default data storage in hive_metastore
"""
# check if the datasets are loaded successfully 
display(dbutils.fs.ls("dbfs:/FileStore/tables/HDS/Week7"))

# use spark to read the datasets
ownership_df = spark.read.csv("dbfs:/FileStore/tables/HDS/Week7/OP_DTL_OWNRSHP_PGYR2023_P01302025_01212025.csv", header=True, inferSchema=True)

recipient_df = spark.read.csv("dbfs:/FileStore/tables/HDS/Week7/OP_CVRD_RCPNT_PRFL_SPLMTL_P01302025_01212025.csv", header=True, inferSchema=True)

research_df = spark.read.csv("dbfs:/FileStore/tables/HDS/Week7/OP_DTL_RSRCH_PGYR2023_P01302025_01212025.csv", header=True, inferSchema=True)

# save these data as tables in default data storage
ownership_df.write.mode("overwrite").saveAsTable("default.op_ownership_2023")
recipient_df.write.mode("overwrite").saveAsTable("default.op_recipient_2023")
research_df.write.mode("overwrite").saveAsTable("default.op_researc_2023")
"""

Out[85]: '\n# check if the datasets are loaded successfully \ndisplay(dbutils.fs.ls("dbfs:/FileStore/tables/HDS/Week7"))\n\n# use spark to read the datasets\nownership_df = spark.read.csv("dbfs:/FileStore/tables/HDS/Week7/OP_DTL_OWNRSHP_PGYR2023_P01302025_01212025.csv", header=True, inferSchema=True)\n\nrecipient_df = spark.read.csv("dbfs:/FileStore/tables/HDS/Week7/OP_CVRD_RCPNT_PRFL_SPLMTL_P01302025_01212025.csv", header=True, inferSchema=True)\n\nresearch_df = spark.read.csv("dbfs:/FileStore/tables/HDS/Week7/OP_DTL_RSRCH_PGYR2023_P01302025_01212025.csv", header=True, inferSchema=True)\n\n# save these data as tables in default data storage\nownership_df.write.mode("overwrite").saveAsTable("default.op_ownership_2023")\nrecipient_df.write.mode("overwrite").saveAsTable("default.op_recipient_2023")\nresearch_df.write.mode("overwrite").saveAsTable("default.op_researc_2023")\n'

Question 1,2,3 which requires "OP_DTL_GNRL_PGYR2023_P01302025_01212025.csv" Data (including Nature of Payments column) can not be loaded in databricks and presumably can not loaded to jupyter notebook. I tried to uzip the files as it was in the discussion, but I could not load the data with my current knowledge. I ain't sure but there seems to be something wrong with that data. Therefore, I am only using the owenership, recipient, and research data and reformulated the original question 1,2,3 in a similar manner that can be done with these tables. Please consider this in mind.

This notebook answers 5 questions about the dataset and show results using spark and sql for comparison.

1. What are the top 10 Terms of Interest with more than $1,000 invested by total number?

In [0]:
from pyspark.sql.functions import col, count

# filter the data by requirements
ownership_df.filter(col("Total_Amount_Invested_USDollars") > 1000) \
    .groupBy("Terms_of_Interest") \
    .agg(count("*").alias("count")) \
    .orderBy(col("count").desc()) \
    .limit(10) \
    .show()

+--------------------+-----+
|   Terms_of_Interest|count|
+--------------------+-----+
| Membership Interest|  157|
|        Common Stock|  111|
|               Stock|   69|
|1.) Value of inte...|   33|
|              shares|   28|
|Interest acquired...|   25|
|Common and prefer...|   23|
|ownership interes...|   23|
|              Equity|   21|
|Limited Liability...|   20|
+--------------------+-----+



In [0]:
%sql
SELECT Terms_of_Interest,
       COUNT(*) AS count
FROM op_ownership_2023
WHERE Total_Amount_Invested_USDollars > 1000
GROUP BY Terms_of_Interest
ORDER BY count DESC
LIMIT 10;

Terms_of_Interest,count
Membership Interest,157
Common Stock,111
Stock,69
1.) Value of interest is a cumulative total calculated by adding year-over-year ownership values per provider.,33
shares,28
Interest acquired as part of milestone payment for acquisition,25
Common and preferred stock,23
ownership interest transferred to HCP as part of terms of acquisition,23
Equity,21
Limited Liability Company Interest,20


* Membership interest, Common stock, and stock are dominating terms of interest that was invested over $1000.
* Comparison of python and sql results are the same.

2. What are the top principal investigators or covered recipients by total number?

In [0]:
# filter the data by requirements
research_df.filter(col("Principal_Investigator_1_Covered_Recipient_Type").isNotNull()) \
    .groupBy("Principal_Investigator_1_Covered_Recipient_Type") \
    .agg(count("*").alias("count")) \
    .orderBy(col("count").desc()) \
    .show(truncate = False)

+-----------------------------------------------+------+
|Principal_Investigator_1_Covered_Recipient_Type|count |
+-----------------------------------------------+------+
|Covered Recipient Physician                    |994641|
|Covered Recipient Non-Physician Practitioner   |2140  |
|United States                                  |6     |
|1306013610                                     |1     |
+-----------------------------------------------+------+



In [0]:
%sql
SELECT Principal_Investigator_1_Covered_Recipient_Type,
       COUNT(*) AS count
FROM op_research_2023
WHERE Principal_Investigator_1_Covered_Recipient_Type IS NOT NULL
GROUP BY Principal_Investigator_1_Covered_Recipient_Type
ORDER BY count DESC;

Principal_Investigator_1_Covered_Recipient_Type,count
Covered Recipient Physician,994641
Covered Recipient Non-Physician Practitioner,2140
United States,6
1306013610,1


* Covered recipient physicans consists the overwhelming proportion of covered recipients.
* Comparison of python and sql results are the same.

3. What are the top 10 product categories or therapeutic areas by total amount of research payments?

In [0]:
from pyspark.sql.functions import sum

# convert Total Amount of Payment US Dollars into string
research_df = research_df.withColumn(
    "Total_Amount_of_Payment_USDollars",
    col("Total_Amount_of_Payment_USDollars").cast("float")
)

# filter the data by requirements
research_df.filter(col("Product_Category_or_Therapeutic_Area_1").isNotNull()) \
    .groupBy("Product_Category_or_Therapeutic_Area_1") \
    .agg(sum("Total_Amount_of_Payment_USDollars").alias("total_amount")) \
    .orderBy(col("total_amount").desc()) \
    .limit(10) \
    .show()

+--------------------------------------+--------------------+
|Product_Category_or_Therapeutic_Area_1|        total_amount|
+--------------------------------------+--------------------+
|                              Oncology|1.0304596049347514E9|
|                              ONCOLOGY| 9.838664319536396E8|
|                            Immunology|1.8717888397264186E8|
|                             Neurology|1.7098073143218708E8|
|                  Infections and In...| 1.589445241275518E8|
|                              VACCINES|1.0753509171267414E8|
|                  Cardiology/Vascul...| 8.555152524288756E7|
|                           BioOncology| 6.102433124171868E7|
|                         Ophthalmology| 5.763378277834958E7|
|                            IMMUNOLOGY| 4.559288536039591E7|
+--------------------------------------+--------------------+



In [0]:
%sql
SELECT Product_Category_or_Therapeutic_Area_1,
       SUM(Total_Amount_of_Payment_USDollars) AS total_amount
FROM op_research_2023
WHERE Product_Category_or_Therapeutic_Area_1 IS NOT NULL
GROUP BY Product_Category_or_Therapeutic_Area_1
ORDER BY total_amount DESC
LIMIT 10;


Product_Category_or_Therapeutic_Area_1,total_amount
Oncology,1.0304596055000001E9
ONCOLOGY,9.838664305599917E8
Immunology,1.8717888408000025E8
Neurology,1.7098073142999992E8
Infections and Infectious Diseases,1.589445241800033E8
VACCINES,1.0753509165000033E8
Cardiology/Vascular Diseases,8.555152519999978E7
BioOncology,6.1024330910000004E7
Ophthalmology,5.763378275999986E7
IMMUNOLOGY,4.559288520999992E7


* Oncology and immunology seems to be funded very well, considering they have two spots respectively in top 10. Oncology also includes BioOncology.
* Comparison of python and sql results are the same.

* The total amount was not clear so we assume that it is Total_Amount_Invested_USDollars.

4. What are the top ten physician specialties by total amount?

In [0]:
# filter the data by requirements
ownership_df.filter(col("Physician_Specialty").isNotNull()) \
    .groupBy("Physician_Specialty") \
    .agg(sum("Total_Amount_Invested_USDollars").alias("total_amount")) \
    .orderBy(col("total_amount").desc()) \
    .limit(10) \
    .selectExpr(
        "Physician_Specialty",
        "ROUND(total_amount, 2) as Total_Invested"
    ) \
    .show(truncate = False)

+----------------------------------------------------------------------------------------+--------------+
|Physician_Specialty                                                                     |Total_Invested|
+----------------------------------------------------------------------------------------+--------------+
|Allopathic & Osteopathic Physicians|Internal Medicine                                   |1.2617592175E8|
|Allopathic & Osteopathic Physicians|Orthopaedic Surgery                                 |1.720776963E7 |
|Allopathic & Osteopathic Physicians|Neurological Surgery                                |9215456.67    |
|Allopathic & Osteopathic Physicians|General Practice                                    |7833554.49    |
|Allopathic & Osteopathic Physicians|Orthopaedic Surgery|Orthopaedic Surgery of the Spine|7774635.44    |
|Allopathic & Osteopathic Physicians|Internal Medicine|Medical Oncology                  |5733256.5     |
|Allopathic & Osteopathic Physicians|Ophthalmo

In [0]:
%sql
SELECT Physician_Specialty,
       ROUND(SUM(CAST(Total_Amount_Invested_USDollars AS FLOAT)), 2) AS Total_Invested_USD
FROM op_ownership_2023
WHERE Physician_Specialty IS NOT NULL
GROUP BY Physician_Specialty
ORDER BY Total_Invested_USD DESC
LIMIT 10;

Physician_Specialty,Total_Invested_USD
Allopathic & Osteopathic Physicians|Internal Medicine,1.2617592175E8
Allopathic & Osteopathic Physicians|Orthopaedic Surgery,1.720776963E7
Allopathic & Osteopathic Physicians|Neurological Surgery,9215456.67
Allopathic & Osteopathic Physicians|General Practice,7833554.49
Allopathic & Osteopathic Physicians|Orthopaedic Surgery|Orthopaedic Surgery of the Spine,7774635.44
Allopathic & Osteopathic Physicians|Internal Medicine|Medical Oncology,5733256.5
Allopathic & Osteopathic Physicians|Ophthalmology,4132615.43
Dental Providers|Dentist,3791925.95
Dental Providers|Dentist|Endodontics,3580273.5
Allopathic & Osteopathic Physicians|Orthopaedic Surgery|Foot and Ankle Surgery,2632503.0


* We can see that various specialties from physicians are invested, specifically for Allopathic & Osteopathic physicians, consisting 8 out of 10.
* Comparison of python and sql results are the same.

%md
5. Who are the top ten physicians by total amount?

In [0]:
from pyspark.sql.functions import concat_ws, round

# filter the data by requirements
ownership_df.filter(col("Total_Amount_Invested_USDollars").isNotNull()) \
    .withColumn("Total_Amount_Invested_USDollars", col("Total_Amount_Invested_USDollars").cast("float")) \
    .withColumn("Physician_Full_Name", concat_ws(" ", col("Physician_First_Name"), col("Physician_Last_Name"))) \
    .groupBy("Physician_Full_Name") \
    .agg(round(sum("Total_Amount_Invested_USDollars"), 2).alias("total_amount")) \
    .orderBy(col("total_amount").desc()) \
    .limit(10) \
    .show(truncate=False)

+-------------------+------------+
|Physician_Full_Name|total_amount|
+-------------------+------------+
|George Lopez       |1.2468128E8 |
|FREDERIC MOLL      |4809215.49  |
|GEORGE DEMETRI     |4664013.5   |
|Charles Goodis     |3580063.5   |
|Benjamin MacLennan |2200000.0   |
|JEROME BARAKOS     |1894999.0   |
|Martha Stark       |1623989.0   |
|SUNIL GUPTA        |1428704.0   |
|Tamir Tawfik       |1345000.0   |
|Curtis Hamann      |1250000.0   |
+-------------------+------------+



In [0]:
%sql
SELECT CONCAT_WS(' ', Physician_First_Name, Physician_Last_Name) AS Physician_Full_Name,
       ROUND(SUM(CAST(Total_Amount_Invested_USDollars AS FLOAT)), 2) AS total_amount
FROM op_ownership_2023
WHERE Total_Amount_Invested_USDollars IS NOT NULL
GROUP BY Physician_Full_Name
ORDER BY total_amount DESC
LIMIT 10;

Physician_Full_Name,total_amount
George Lopez,1.2468128E8
FREDERIC MOLL,4809215.49
GEORGE DEMETRI,4664013.5
Charles Goodis,3580063.5
Benjamin MacLennan,2200000.0
JEROME BARAKOS,1894999.0
Martha Stark,1623989.0
SUNIL GUPTA,1428704.0
Tamir Tawfik,1345000.0
Curtis Hamann,1250000.0


* The top physicia who were invested the most was George lopez.
* Comparison of python and sql results are the same.